In [39]:
from docx import Document
from docx.shared import Inches
from docx.shared import Pt
from datetime import date

# 문서 불러오기
doc = Document('contract_form.docx')
para = doc.paragraphs

# 기본 문서 꾸미기 
para[6].text = "주소 : 성남시 수정구 수정 1동 1번지"
para[7].text = "상호 : 거니전자"
para[8].text = ''
run = para[8].add_run("성명 : 이 거 니")
run.font.size = Pt(15)
run.bold = True

# 텍스트 파일 로드와 내용 저장
order_list = []
file = open("list.txt", "r", encoding='UTF-8')
contents = file.read().split('\n')

# 저장된 텍스트 내용 뿌리기 시작 
issue_date = contents.pop(0)
deli_date = contents.pop(0)
title = contents.pop(0)

# 첫 3개의 데이터를 소비한 뒤 이후 물건과 물건 정보를 4개씩 받기
while len(contents) > 0 :
    product = {'product_name':'', 'product_qty':'', 'product_original_pr':'', 'product_selling_pr':''}
    product['product_name'] = contents.pop(0)
    product['product_qty'] = contents.pop(0)
    product['product_original_pr'] = contents.pop(0)
    product['product_selling_pr'] = contents.pop(0)

    # 물건 하나당 딕셔너리 형태로 만들어 리스트에 넣기 
    order_list.append(product)

order_table = doc.tables[0]

total_qty = 0
total_o_p = 0
total_s_p = 0

# 테이블에 물건 정보 뿌려주고 밑에서는 총합 계산
for i, order in enumerate(order_list):
    order_table.rows[i+1].cells[0].paragraphs[0].add_run(order['product_name'])
    order_table.rows[i+1].cells[1].paragraphs[0].add_run(order['product_qty'])
    order_table.rows[i+1].cells[2].paragraphs[0].add_run(order['product_original_pr'])
    order_table.rows[i+1].cells[3].paragraphs[0].add_run(order['product_selling_pr'])
    
    total_qty += int(order['product_qty'])
    total_o_p += int(order['product_original_pr'])
    total_s_p += int(order['product_selling_pr'])

# 총합 입력
order_table.rows[6].cells[1].paragraphs[0].add_run(str(total_qty))
order_table.rows[6].cells[2].paragraphs[0].add_run(str(total_o_p))
order_table.rows[6].cells[3].paragraphs[0].add_run(str(total_s_p)) 

# 텍스트에서 읽어온 계약 건명 넣기
run = para[11].add_run(title)
run.font.size = Pt(11)
run.bold = True

# 계산된 총 금액 넣기
run = para[12].add_run(str(total_s_p))
run.font.size = Pt(11)
run.bold = True

# 오늘 날짜 저장
today = date.today()

# 만약 매매 계약 날짜가 오늘이라면, 오늘 날짜 입력 
if issue_date == "오늘":
    para[4].text = today.strftime("%Y년 %m월 %d일")
else:
    para[4].text = issue_date

# 만약 납품 날짜가 오늘이라면, 오늘 날짜 입력 
if deli_date == "오늘":
    exp_str = today.strftime("%Y년 %m월 %d일")
# 납품 날짜가 내일이라면 내일 날짜 입력 
elif deli_date == "내일":
    exp_str = (today + datetime.timedelta(days=1)).strftime("%Y년 %m월 %d일")
else:
    exp_str = deli_date

# 납품 문자열 꾸며주기
para[18].text = ''
run = para[18].add_run("1." + exp_str + "까지 지정한 장소에 납품할 것이며, 그 납품 중 검사 불합격품이 있을 때에는 지정기일까지 교환하겠음.")
run.font.size = Pt(11)

# 새로운 문서 파일로 저장 
doc.save('result.docx') 
